# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [169]:
!cat publications-upd.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	link	teaser
2020-01-21	Atomic limit and inversion-symmetry indicators for topological superconductors	"Phys. Rev. Research 2, 013064"	"Symmetry indicators allow to identify topological crystalline insulators using symmetry-group analysis of their Bloch states. An extension of this approach to superconducting systems requires defining an appropriate atomic limit for superconductors. Here, we introduce such a notion of atomic limit and derive a symmetry indicator for inversion-symmetric superconductors. This indicator allows for a refined topological classification including higher-order phases for systems in the superconducting phase with and without time-reversal symmetry. We further elucidate their bulk-boundary correspondence using Dirac surface theories. This indicator is well suited for a search of topological superconductors using first-principles calculations.
"	"A. Skurativska, T. Neupert, and M. H. Fischer"	sym-indicators

## Import pandas

We are using the very handy pandas library for dataframes.

In [175]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [176]:
publications = pd.read_csv("publications-upd.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,link,teaser
0,2020-01-21,Atomic limit and inversion-symmetry indicators...,"Phys. Rev. Research 2, 013064",Symmetry indicators allow to identify topologi...,"A. Skurativska, T. Neupert, and M. H. Fischer",sym-indicators,/files/sym-indicators.pdf,https://journals.aps.org/prresearch/abstract/1...,/images/sym-indicators-teaser.png
1,2021-08-09,Spin response and topology of a staggered-Rash...,"Phys. Rev. Research 3, 033133",Inversion symmetry is a key symmetry in unconv...,"A. Skurativska, M. Sigrist, and M. H. Fischer",rashba,/files/rashba.pdf,https://journals.aps.org/prresearch/abstract/1...,/images/rashba-teaser.png
2,2021-07-02,Flat bands with fragile topology through super...,"Phys. Rev. Research 3, L032003",Magic-angle twisted bilayer graphene has recei...,"A. Skurativska, S. S. Tsirkin, F. D. Natterer,...",flat-band,/files/flat-bands.pdf,https://journals.aps.org/prresearch/abstract/1...,/images/flat-bands-teaser.png
3,2021-09-28,Exceptional topological insulators,"Nat. Comm. 12, 5681",We introduce the exceptional topological insul...,"M. M. Denner, A. Skurativska, F. Schindler, M....",eti,/files/eti.pdf,https://www.nature.com/articles/s41467-021-259...,/images/eti-teaser.png
4,2023-06-15,Robust spin polarization of Yu-Shiba-Rusinov s...,"Phys. Rev. B 107, 224507",Yu-Shiba-Rusinov states are subgap bound state...,"A. Skurativska, J. Ortuzar, D. Bercioux, F. S....",ysr,/files/ysr.pdf,https://journals.aps.org/prb/abstract/10.1103/...,/images/ysr-teaser.png


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [177]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [180]:
import os
for row, item in publications.iterrows():
    
    print(item)

    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]

    print('print',year)
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"

    #this line was added to add link to paper online
    if len(str(item.link)) > 5:
        md += "\nexternallink: '" + item.link + "'"

    if len(str(item.teaser)) > 5:
        md += "\nimage: '" + item.teaser + "'"
    

    
    md += "\n---"

        
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    #if len(str(item.paper_url)) > 5:
    #    md += "\n[PDF](" + item.paper_url + ")\n" 
        
    #md += "\nRecommended citation: " + item.citation
    
    #md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

pub_date                                            2020-01-21
title        Atomic limit and inversion-symmetry indicators...
venue                            Phys. Rev. Research 2, 013064
excerpt      Symmetry indicators allow to identify topologi...
citation         A. Skurativska, T. Neupert, and M. H. Fischer
url_slug                                        sym-indicators
paper_url                            /files/sym-indicators.pdf
link         https://journals.aps.org/prresearch/abstract/1...
teaser                       /images/sym-indicators-teaser.png
Name: 0, dtype: object
print 2020
pub_date                                            2021-08-09
title        Spin response and topology of a staggered-Rash...
venue                            Phys. Rev. Research 3, 033133
excerpt      Inversion symmetry is a key symmetry in unconv...
citation         A. Skurativska, M. Sigrist, and M. H. Fischer
url_slug                                                rashba
paper_url            

These files are in the publications directory, one directory below where we're working from.

In [179]:
!ls ../_publications/

2020-01-21-sym-indicators.md 2021-09-28-eti.md
2021-07-02-flat-band.md      2023-06-15-ysr.md
2021-08-09-rashba.md


In [168]:
#!cat ../_publications/2009-10-01-paper-title-number-1.md